# Transcribe Recordings
paid service [Goodtape](http://goodtape.io) will be used for the transcriptions

## Imports

In [27]:
%reload_ext autoreload
%autoreload 2

In [2]:
import requests
import pandas as pd
import os
import docx

# local imports
from lib_henryk.logger import *
from lib_henryk.config import *
from lib_henryk.utils import *

from lib_henryk.recordings import transcription

## Params

In [3]:
# load api keys
from dotenv import load_dotenv
_ = load_dotenv()

# retrieve new token by opening 'https://webhook.site/' in incognito mode
# and picking up the token from the URL
WEBHOOKS_TOKEN_ID = 'f7477993-66da-4747-b9c8-6bc5c302ef3c'
GOODTAPE_API_KEY = os.getenv("GOODTAPE_API_KEY")

## Load Dataset

In [4]:
df = pd.read_csv(FILE_AUDIO_STATS_CSV)
df_transcriptions = df.copy()
df_transcriptions['transcription_id'] = ''

In [20]:
# load sync dataframe
if os.path.exists(FILE_TRANSCRIPTION_CSV):
    print(f'transcription log exists, reading it from {FILE_TRANSCRIPTION_CSV}')
    df_transcriptions = pd.read_csv(FILE_TRANSCRIPTION_CSV)

transcription log exists, reading it from ../../data/processed/henryk_audio_transcription.csv


## Run Transcriptions

In [21]:
# perform transcriptions
transcription.submit_transcriptions_goodtape(
    df=df_transcriptions, 
    webhooks_token_id=WEBHOOKS_TOKEN_ID,
    goodtape_api_key=GOODTAPE_API_KEY,
    path_recordings=DIR_WIADOMOSCI_DO_HENRYCZKA,
    start=0, stop=500,
    verbose=False
)

submitted 0 recordings to the transcription service GoodTape


## Fetch Transcriptions JSONs

In [7]:
# need to wait until transcriptions are available
exit_cell()

StopExecution: stopped

In [28]:
# retrieve goodtape server responses from webhooks
transcription.retrieve_responses_goodtape_via_webhooks(
    df=df_transcriptions,
    webhooks_token_id=WEBHOOKS_TOKEN_ID, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON
)

# process json to text
transcription.process_transcriptions_json(
    df=df_transcriptions, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON, 
    path_transcriptions_txt=DIR_TRANSCRIPTIONS_TXT,
    path_transcriptions_doc=DIR_TRANSCRIPTIONS_DOC,
    to_doc=True,
    to_txt=True,
    verbose=False,
)

# process json to doc

found 0 requests to fetch
0 transcriptions were retrieved
processing 498 transcriptions |██████████████████████████████████████████████████| 100.0% done.
498 new transcriptions were processed, there are 498 txt and 498 doc transcriptions available


## Cleanup missing requests
- transcription ids for the requests that were lost will be removed, only those that were processed will remain

In [31]:
# cleanup stale transcription ids - but work on a copy of the log
df_transcriptions_to_write = transcription.get_verified_transcriptions(
    df=df_transcriptions, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON, 
    path_transcriptions_txt=DIR_TRANSCRIPTIONS_TXT,
    path_transcriptions_doc=DIR_TRANSCRIPTIONS_DOC,
)

# save transcription log
df_transcriptions_to_write.to_csv(FILE_TRANSCRIPTION_CSV, index=False)

verifying 498 transcriptions |██████████████████████████████████████████████████| 100.0% done.
there are 498 processed and valid transcriptions
*** all 498 recordings were transcribed ***
